## 本地构建镜像并推导ECR

In [ ]:
from utils import *

In [ ]:
image = build_and_push(tag='ml-ecs-demo', dockerfile_path='.', ecr_repo_name='ml-ecs-demo')

In [ ]:
image

## 上传代码至s3

In [ ]:
sagemaker_session = sagemaker.session.Session()
bucket = sagemaker_session.default_bucket()

prefix='ecs-demo/train'

In [ ]:
code_files = get_file_path('./train_code')

# 压缩代码文件为tar.gz
create_tar_file(code_files, "sourcedir.tar.gz")
# 代码上传到s3上路径
sources = sagemaker_session.upload_data("sourcedir.tar.gz", bucket, os.path.join(prefix, 'code'))
sources

In [ ]:
train_path =  sagemaker_session.upload_data("dataset/train.csv", bucket, os.path.join(prefix, 'data'))
train_path

In [ ]:
model_path = 's3://{}/{}/model/'.format(bucket, prefix)
model_path

# 创建 Task Definition

ECS(Elastic Container Services) 是 AWS 上的服务，可以运行Docker。

ECS的运行方式如下

1. 创建TaskDefinition：指定cpu，内存和ecr镜像地址，执行的命令，可以跑多个container
2. 在运行时，通过boto3调用时，选定一个集群，并设置安全组和启动方式


基于无服务的ECS运行需要下列条件：

- 启动方式为Fargate
- 运行时要选定一个Cluster，需要提前创建
- 有一个CloudWatch Logs的log group，用于存储运行log
  - 如果是手动在console上创建TaskDefinition，会自动创建log group
  - 如果是通过boto3创建的，需要手动创建log group

所以，通过boto3创建task definition，需要

- 创建 CloudWatch log group
- 创建 Task Definition
- 启动 Task Definition

## 创建log group

需要设定的参数：

- `log_group_name`：CloudWatch上的log group名字，用于创建log group


In [ ]:
import boto3
import botocore
log_group_name = '/ecs/ml-ecs-demo' # CloudWatch上的log group名字，用于创建log group
client = boto3.client('logs')

try:
    response = client.create_log_group(
        logGroupName=log_group_name,
    )
    print(response)
except Exception as e:
    if 'ResourceAlreadyExistsException' in str(e):
        print('Log Group already existed')
    else:
        print(e)

## 创建Task Definition

接下来，开始创建 Task Definition：

需要指定下列参数：

- `task_name`：TaskDefinition的名字
- `log_group_name`：Task log地址
- `stream_prefix`：log group流的prefix
- `cmd`：入口命令，Docker实际执行的命令，是一个list，把空格分割用list代替，例如`echo 123`为`['echo', '123']`
- `image`：镜像在ECR上的路径
- `task_role_arn`和`execution_role_arn`：区别暂时不清楚，先用同一个，要给AmazonECSTaskExecutionRolePolicy
- `cpu`：cpu数，每1024为1核，最高4核，必须是字符串
- `memory`：memory 30720为30G，最高30G，必须是字符串

In [ ]:
import boto3

task_name = 'ml-ecs-demo' # TaskDefinition的名字

log_group_name = '/ecs/ml-ecs-demo' # CloudWatch上的log group名字，用于创建log group
region = 'cn-northwest-1' # log group所在的region
stream_prefix = 'ecs' # log group流的prefix

cmd = ['./cmd.sh', sources, train_path, model_path ] # CMD命令,Docker实际执行的命令
image = image # 镜像在ECR上的路径

task_role_arn = 'arn:aws-cn:iam::xxx:role/xxx' # Task Role，需要拥有AmazonECSTaskExecutionRolePolicy
execution_role_arn = 'arn:aws-cn:iam::xxx:role/xxx' # Task Role，需要拥有AmazonECSTaskExecutionRolePolicy

cpu = str(4096) # cpu数，每1024为1核，最高4核
memory = str(30720) # memory 30720为30G，最高30G

client = boto3.client('ecs')

response = client.register_task_definition(
    family=task_name,
    taskRoleArn=task_role_arn,
    executionRoleArn=execution_role_arn,
    networkMode='awsvpc',
    containerDefinitions=[
        {
            'name': 'container1',
            'image': image,

            'command': cmd,
            
            'logConfiguration': {
                'logDriver': 'awslogs',
                'options': {
                    'awslogs-region' : region,
                    'awslogs-group' : log_group_name,
                    'awslogs-stream-prefix': stream_prefix
                }
            },

        }, 
    ], 
    requiresCompatibilities=[
        'FARGATE',
    ],
    cpu=cpu,
    memory=memory
)

# 创建ECS Cluster

In [ ]:
import boto3

client = boto3.client('ecs')

response = client.create_cluster(
    clusterName='clutser-name',
    configuration={
        'executeCommandConfiguration': {
            'logging': 'DEFAULT',
        }
    },
    capacityProviders=[
        'FARGATE_SPOT',
    ]
)

# 创建Lambda启动Task Definition

In [ ]:
!mkdir lambda_train

In [ ]:
%%writefile lambda_train/lambda_function.py

import json
import boto3

def lambda_handler(event, context):
    client = boto3.client('ecs')
    
    cluster = 'cluster-name'
    task_name = 'ml-ecs-demo'
    subnets = ['subnet-xxx']
    security_groups = ['sg-xxx']
    
    
    response = client.run_task(
          cluster=cluster,
          taskDefinition=task_name,
          count=1,
          launchType='FARGATE',
          networkConfiguration={
              'awsvpcConfiguration': {
                  'subnets': subnets,
                  'securityGroups': security_groups,
                  'assignPublicIp': 'ENABLED'
              }
          }
    )

In [ ]:
%%sh
cd lambda_train

zip -r9 function.zip lambda_function.py

# aws lambda delete-function --function-name $lambda_function_name

aws lambda create-function --function-name ml-ecs-demo \
    --runtime python3.8 \
    --zip-file fileb://function.zip \
    --handler lambda_function.lambda_handler \
    --role arn:aws-cn:iam::xxx:role/xxx-lambda-role \
    --timeout 600